In [1]:
import json
import pymongo
import pandas as pd
from collections import defaultdict

In [2]:
def name_clean(member_name):
    member_name = member_name.strip()
    if 'Sh.' in member_name:
        member_list = member_name.split('Sh.')
        name = 'Sh.'+" "+member_list[-1]+" "+' '.join(member_list[:-1])
    elif 'Smt.' in member_name:
        member_list = member_name.split('Smt.')
        name = 'Smt.'+" "+member_list[-1]+" "+' '.join(member_list[:-1])
    elif ',' in member_name:
        name_list = member_name.split(',')
        if len(name_list) >= 2:
            name = name_list[1].strip()+" "+name_list[0].strip()
            name = name.replace("  "," ").replace('\xa0','')
    else:
        name = member_name
    return name

def get_gender(name):
    title = name.split(" ")[0]
    female_titles = [
        'Begam',
        'Begum',
        'Devi',
        'Dr.(Kum.)',
        'Dr.(Smt.)',
        'Dr.(Smt.)Chellamalla',
        'Her',
        'Km.',
        'Kum.',
        'Kumari',
        'Maharani',
        'Ms.',
        'Prof.(Smt.)',
        'Rajkumari',
        'Rajmata',
        'Rajmata(Jodhpur)',
        'Rani',
        'Sadhvi',
        'Sardarni',
        'Smt',
        'Smt.',
        'Sushree'
    ]
    male_titles = [
        '(Pandit)',
        'Acharya',
        'Adv.',
        'Advocate',
        'Baba',
        'Babu',
        'Capt.',
        'Capt.(Retd.)',
        'Captain',
        'Ch.',
        'Chaudhari',
        'Chaudhary',
        'Chaudhri',
        'Chaudhry',
        'Choudari',
        'Choudhary',
        'Choudhury',
        'Chowdhary',
        'Chowdhry',
        'Chowdhury',
        'Col.',
        'Dr.Maharaja',
        'Father',
        'General',
        'H.H.',
        'H.H.Maharaja',
        'Haji',
        'Kazi',
        'Kh.',
        'Kunwar',
        'Lala',
        'Lt.',
        'Lt.Col.',
        'Mahant',
        'Maharajkumar',
        'Maj',
        'Maj.',
        'Maj.Gen.',
        'Major',
        'Major-General',
        'Maulana',
        'Mohammad',
        'Mohd.',
        'Motilal',
        'Mulla',
        'Pandit',
        'Purohit',
        'Qazi',
        'Raja',
        'Rt.',
        'Sardar',
        'Seth',
        'Shaikh',
        'Shri',
        'Shriman',
        'Swami',
        'Syed',
        'Thakur',
        'Thiru',
        'Vaidya',
        'Visharad'
    ]
    if title in female_titles:
        return "Female"
    elif title in male_titles:
        return "Male"
    else:
        return "Not Available"

In [4]:
config_file = open("../config.cfg")
config = json.load(config_file)
client = pymongo.MongoClient(config["mongodb_uri"])
db = client[config["database"]]
client2 = pymongo.MongoClient(config["mongodb_uri2"])
db2 = client2[config["database"]]

In [5]:
old_members_original = db2["ls_all_member_urls"]
old_members_copy = db["archive_ls_members"]

In [6]:
data = []
for row in old_members_original.find({}):
    data.append(row)

In [60]:
df = pd.DataFrame(columns=["_id","name","gender",
                            "party","constituency","state",
                            "session","dob","birth_place",
                            "marital_status","sons","daughters",
                            "education","profession"])

In [54]:
rename_cols = {
    "ID" : "_id",
    "Name" : "name",
    "Gender" : "gender",
    "Party" : "party",
    "Constituency" : "constituency",
    "State" : "state",
    "Sessions" : "session",
    "sessions" : "session",
    "DOB" : "dob",
    "Birth_place" : "birth_place",
    "Birth_Place" : "birth_place",
    "Marital_status" : "marital_status",
    "Marital_Status" : "marital_status",
    "Sons" : "sons",
    "Daughters" : "daughters",
    "Education" : "education",
    "Profession" : "profession",
    "URL" : "url"
}

In [61]:
for row in data:
    this_row = {}
    this_row = defaultdict(lambda: "Not Available" , this_row)
    keys = list(row.keys())
    for key in keys:
        if key == '_id':
            continue
        elif key == 'Name':
            this_row[rename_cols[key]] = name_clean(row[key])
            this_row["gender"] = get_gender(name_clean(row[key]))
        elif key == 'Details' and 'Education' not in keys:
            this_row["education"] = row["Details"]
        elif key == 'Education':
            if "Details" not in keys:
                this_row["education"] = row["Education"]
            else:
                this_row["education"] = row["Education"] + " " + row["Details"]
        elif key != 'Details':
            this_row[rename_cols[key]] = row[key]
    df = df.append(dict(this_row), ignore_index=True)

In [62]:
df = df.fillna("Not Available")

In [65]:
records = json.loads(df.T.to_json()).values()

In [66]:
# old_members_copy.insert_many(records)